In [1]:
import tensorflow as tf
import tensorflow.keras as keras
print(tf.__version__)
print(keras.__version__)

2.5.0
2.5.0


In [2]:
n_outputs = 3

In [3]:
import numpy as np
import os
from os import listdir 
from os.path import isfile, join
import cv2

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.utils import to_categorical
# from sklearn.model_selection import train_test_split

import random
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

In [4]:
# Searching training data

label_folder = []
total_size = 0
data_path = r"C:\Users\ESA LAB\Desktop\CNN_FOG_711083104\CNN_FOG - 1 - 50%OL-spilit_patient\dataset\RP"

#os.walk() generates the file names(dirpath, dirnames, filenames) 
#in a directory tree by walking the tree either top-down or bottom-up.
for root, dirts, files in os.walk(data_path): 
    for dirt in dirts:
        label_folder.append(dirt)
    total_size += len(files)

    
print("found",total_size,"files.")
print("folder:",label_folder)

found 26986 files.
folder: ['0', '1', '2']


In [5]:
#Load training images

base_x_train = []
base_y_train = []

for i in range(len(label_folder)):
    labelPath = data_path+r'\\'+label_folder[i]
    
    #listdir() returns a list containing the names of the entries in the directory given by path.
    #isfile() is used to check whether the specified path is an existing regular file or not.
    FileName = [f for f in listdir(labelPath) if isfile(join(labelPath, f))]
    
    for j in range(len(FileName)):
        path = labelPath+r'\\'+FileName[j]
        
        #use cv2.imread read image.
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        
        base_x_train.append(img)
        base_y_train.append(int(label_folder[i]))


print(np.array(base_x_train).shape)
print(np.array(base_y_train).shape)

(26986, 72, 72, 3)
(26986,)


In [6]:
# normalization the data, every value is between 0 and 1

base_x_train = np.array(base_x_train) 
base_y_train = np.array(base_y_train)


base_x_train = base_x_train.astype('float')/255.0

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), padding = "same", activation='relu', input_shape=(72, 72, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3, 3), padding = "same", activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(64, (3, 3), padding = "same", activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3, 3), padding = "same", activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(128, (3, 3), padding = "same", activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, (3, 3), padding = "same", activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(n_outputs, activation = "softmax")
])
	

In [8]:
# using Stratified KFold cross-validation to split datas
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
skf.get_n_splits(base_x_train, base_y_train)

5

In [9]:
epoch = 60
batch_size = 100


for i, (train, test) in enumerate(skf.split(base_x_train, base_y_train)):
    print("==> Fold #%d" % i)

    x_train, x_val = base_x_train[train], base_x_train[test]
    y_train, y_val = base_y_train[train], base_y_train[test]

    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)

    model.compile(
        loss="categorical_crossentropy", 
        optimizer="adam", 
        metrics=["accuracy"]
    )
    history = model.fit(
        x_train, 
        y_train, 
        epochs=epoch, 
        batch_size=batch_size,
        validation_data=(x_val, y_val)
    )

==> Fold #0
Epoch 1/60
216/216 [==============================] - 12s 29ms/step - loss: 0.9233 - accuracy: 0.6650 - val_loss: 0.4217 - val_accuracy: 0.8957
Epoch 2/60
216/216 [==============================] - 5s 25ms/step - loss: 0.3179 - accuracy: 0.8912 - val_loss: 0.3646 - val_accuracy: 0.8959
Epoch 3/60
216/216 [==============================] - 5s 25ms/step - loss: 0.2760 - accuracy: 0.8958 - val_loss: 0.6889 - val_accuracy: 0.8957
Epoch 4/60
216/216 [==============================] - 5s 24ms/step - loss: 0.2626 - accuracy: 0.8979 - val_loss: 0.2533 - val_accuracy: 0.8974
Epoch 5/60
216/216 [==============================] - 5s 24ms/step - loss: 0.2533 - accuracy: 0.9019 - val_loss: 0.2469 - val_accuracy: 0.8976
Epoch 6/60
216/216 [==============================] - 5s 24ms/step - loss: 0.2463 - accuracy: 0.9052 - val_loss: 0.2731 - val_accuracy: 0.9013
Epoch 7/60
216/216 [==============================] - 5s 25ms/step - loss: 0.2391 - accuracy: 0.9054 - val_loss: 0.4298 - val_acc

In [10]:
model.save("my_model.hdf5")

In [11]:
from sklearn.metrics import confusion_matrix,accuracy_score,f1_score,roc_auc_score,recall_score,precision_score

In [12]:
# Searching training data

label_folder = []
total_size = 0
data_path = r"C:\Users\ESA LAB\Desktop\CNN_FOG_711083104\CNN_FOG - 1 - 50%OL-spilit_patient\dataset\RPtest"

#os.walk() generates the file names(dirpath, dirnames, filenames) 
#in a directory tree by walking the tree either top-down or bottom-up.
for root, dirts, files in os.walk(data_path): 
    for dirt in dirts:
        label_folder.append(dirt)
    total_size += len(files)

    
print("found",total_size,"files.")
print("folder:",label_folder)

found 6451 files.
folder: ['0', '1', '2']


In [13]:
#Load training images

x_test = []
y_test = []

for i in range(len(label_folder)):
    labelPath = data_path+r'\\'+label_folder[i]
    
    #listdir() returns a list containing the names of the entries in the directory given by path.
    #isfile() is used to check whether the specified path is an existing regular file or not.
    FileName = [f for f in listdir(labelPath) if isfile(join(labelPath, f))]
    
    for j in range(len(FileName)):
        path = labelPath+r'\\'+FileName[j]
        
        #use cv2.imread read image.
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        
        x_test.append(img)
        y_test.append(int(label_folder[i]))


print(np.array(x_test).shape)
print(np.array(y_test).shape)

(6451, 72, 72, 3)
(6451,)


In [14]:
# normalization the data, every value is between 0 and 1

x_test = np.array(x_test) 
y_test = np.array(y_test)


x_test = x_test.astype('float')/255.0

In [15]:
y_test = to_categorical(y_test)
y_pred = model.predict(x_test)

y_test = np.argmax(y_test, axis=1)
y_pred = np.argmax(y_pred, axis=1)

confus = confusion_matrix(y_test, y_pred, labels=None, sample_weight=None)

In [16]:
print(confus)

[[5096  796    3]
 [ 224  282    1]
 [  29   20    0]]


In [17]:
accuracy = accuracy_score(y_test, y_pred)*100
print(accuracy)

83.36691985738646
